In [1]:
# import fluidsynth
import glob
import numpy as np
import pandas as pd
import pretty_midi
import collections
from pathlib import Path
from IPython import display

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

%matplotlib inline
from matplotlib import pyplot as plt

recreate the model that was used for training - which is located in  `./baseline_1/train_baseline.py`

In [7]:
from baseline_1.train_baseline import PianoRollLSTM
from right_chords.train import ChordLSTM, FullChord, get_preprocessed_files
        
# model = PianoRollLSTM(separate=False)
model = ChordLSTM()

load the model state and it put it in eval mode

In [5]:
model_weights_file = 'right_chords/model_weights_iter1000.pth'
model.load_state_dict(torch.load(model_weights_file))
model.eval()

ChordLSTM(
  (lstm): LSTM(1848, 231, batch_first=True)
  (predict_layer): Sequential(
    (0): Linear(in_features=231, out_features=1848, bias=True)
    (1): Softmax(dim=2)
  )
)